In [62]:
import pdfplumber
import pandas
import re

In [63]:
# load pdf file

filenames = ['2 jawa-bali.pdf']

pdf = pdfplumber.open(filenames[0])

In [74]:
# find city name
def findCityName(page):
    text = page.extract_text().split("\n")[0]
    position = text.find("Produk Domestik Regional Bruto")

    cityName = text[position+len("Produk Domestik Regional Bruto"):].strip()

    return cityName

In [84]:
# extracting tables to dataframe

def extractDataframe(page):
    # collect x-coordinates to split text because there aren't any vertical lines   
    xCoordinates = [353, 396, 438, 480, 522, 554]
    
    tables = page.extract_table(table_settings={
        "vertical_strategy": "explicit",
        "explicit_vertical_lines": xCoordinates,
        "horizontal_strategy": "text",
    })

    # find where the data start
    def findIndex(dataList):
        findList = ['(2)', '(3)', '(4)', '(5)', '(6)']

        for index, sublist in enumerate(dataList):
            if sublist == findList:
                index = index + 2
                print(f"data start from index: {index}")
                return index

    # convert extraction result into dataframe
    index = findIndex(tables)
    df = pandas.DataFrame(tables[index:], columns=["2019", "2020", "2021", "2022", "2023"])

    return df

In [93]:
# data cleaning & re-formating

def dataCleaning(value):
    # check if value is negative
    if '(' in value or ')' in value:
        value = '-' + value.replace('(', '').replace(')', '')
    else:
        # remove non numeric
        value = re.sub(r'[^\d,.-]', '', value)

    value = value.replace('.', '').replace(',', '.')
    return value

def tableCleaning(df, cityName):
    # to remove empty value we need to replace "" to NaN
    df = df.replace("", pandas.NA)
    df = df.dropna(how='any')

    # remove unnecessary rows
    df = df[~df["2019"].isin(["2019","(2)", "P", "PD", "2019\n(2)"])]

    # data cleaning
    df = df.map(dataCleaning)

    # adding new columns
    columnNames = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
                   'M,N', 'O', 'P', 'Q', 'R,S,T,U', 'GRDP Total', 'GRDP/Capita',
                   'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
                   'M,N', 'O', 'P', 'Q', 'R,S,T,U', 'GRDP Total', 'GRDP Growth Rate(%)',
                  ]
    
    # checking rows length for validation
    rowsLength = len(df)
    print(f"Current rows found: {rowsLength} in {cityName}")
    midpoint = rowsLength//2
    
    
    # new columns for price type
    marketPriceList = ["Current Market Prices" if i < midpoint else "2010 Constant Market Prices" for i in range(rowsLength)]
    
    df["City"] = cityName
    df["Industry Code"] = columnNames
    df["Market Price Type"] = marketPriceList

    # reshape table
    df = df.melt(id_vars=["Industry Code", "City", "Market Price Type"], var_name="Year", value_name="GRDP")

    return df    

In [94]:
# container for all dataframe
results = []

# identify which page has table
for page in pdf.pages:    
    tables = page.find_tables()
    
    if len(tables) == 2:
        cityName = findCityName(page)

        # validate city name
        if len(cityName) > 20:
            continue
        print(f"processing data for {cityName}")
        
        df = extractDataframe(page)
        df = tableCleaning(df, cityName)
        
        results.append(df)

processing data for Kab. Kep. Seribu
data start from index: 11
Current rows found: 38 in Kab. Kep. Seribu
processing data for Kota Jakarta Selatan
data start from index: 11
Current rows found: 38 in Kota Jakarta Selatan
processing data for Kota Jakarta Timur
data start from index: 11
Current rows found: 38 in Kota Jakarta Timur
processing data for Kota Jakarta Pusat
data start from index: 11
Current rows found: 38 in Kota Jakarta Pusat
processing data for Kota Jakarta Barat
data start from index: 11
Current rows found: 38 in Kota Jakarta Barat
processing data for Kota Jakarta Utara
data start from index: 11
Current rows found: 38 in Kota Jakarta Utara
processing data for Kab. Bogor
data start from index: 11
Current rows found: 38 in Kab. Bogor
processing data for Kab. Sukabumi
data start from index: 11
Current rows found: 38 in Kab. Sukabumi
processing data for Kab. Cianjur
data start from index: 11
Current rows found: 38 in Kab. Cianjur
processing data for Kab. Bandung
data start from

In [95]:
# combine all df
dfConcat = pandas.concat(results)

# save to csv
dfConcat.to_csv("result.csv")